# Python 效能基礎 - GIL、多執行緒與 Benchmarking

## 學習目標

1. 理解 Python GIL（Global Interpreter Lock）
2. 了解 numpy 的內建多執行緒（BLAS/OpenMP）
3. 學習正確的 benchmarking 方法
4. 檢查和設定多執行緒環境

## 為什麼這很重要？

你有 32 執行緒的 CPU，但如果不理解 Python 的執行模型，
可能只用到了 1 個核心的效能！

In [ ]:
import numpy as np
import time
import threading
import multiprocessing
import os

print(f"CPU cores available: {multiprocessing.cpu_count()}")
print(f"NumPy version: {np.__version__}")

---

## 第一部分：Python GIL（Global Interpreter Lock）

### 1.1 GIL 是什麼？

**GIL** 是一個互斥鎖（mutex），它保護 Python 物件的存取，
**一次只允許一個執行緒執行 Python bytecode**。

```
╔═══════════════════════════════════════════════════╗
║              Python Interpreter                    ║
╠═══════════════════════════════════════════════════╣
║                      GIL                          ║
║                       │                           ║
║         ┌─────────────┼─────────────┐             ║
║         ▼             ▼             ▼             ║
║    Thread 1      Thread 2      Thread 3          ║
║    (running)     (blocked)     (blocked)         ║
╚═══════════════════════════════════════════════════╝

同一時間只有一個 thread 可以執行 Python 程式碼！
```

### 1.2 為什麼有 GIL？

- Python 使用 reference counting 做記憶體管理
- 多執行緒同時修改 reference count 會造成 race condition
- GIL 是最簡單的解決方案（雖然犧牲了並行性）

In [ ]:
# 實驗：GIL 對純 Python 計算的影響

def pure_python_work(n):
    """CPU-bound 的純 Python 迴圈"""
    total = 0
    for i in range(n):
        total += i * i
    return total

N = 5_000_000

# 單執行緒
start = time.time()
result1 = pure_python_work(N)
result2 = pure_python_work(N)
time_sequential = time.time() - start
print(f"Sequential (2 tasks): {time_sequential:.3f}s")

# 多執行緒（使用 threading）
def thread_worker(n, results, idx):
    results[idx] = pure_python_work(n)

results = [None, None]
start = time.time()
t1 = threading.Thread(target=thread_worker, args=(N, results, 0))
t2 = threading.Thread(target=thread_worker, args=(N, results, 1))
t1.start()
t2.start()
t1.join()
t2.join()
time_threaded = time.time() - start
print(f"Threaded (2 threads): {time_threaded:.3f}s")

print(f"\nSpeedup: {time_sequential / time_threaded:.2f}x")
print("\n觀察：使用 threading 並沒有加速，甚至可能更慢！")
print("這是因為 GIL 不允許真正的並行執行 Python bytecode。")

In [ ]:
# 實驗：用 multiprocessing 繞過 GIL

def mp_worker(n):
    return pure_python_work(n)

# multiprocessing 使用獨立的 process，每個有自己的 GIL
start = time.time()
with multiprocessing.Pool(processes=2) as pool:
    results = pool.map(mp_worker, [N, N])
time_multiprocess = time.time() - start
print(f"Multiprocessing (2 processes): {time_multiprocess:.3f}s")

print(f"\nSpeedup vs sequential: {time_sequential / time_multiprocess:.2f}x")
print("\n觀察：multiprocessing 真的能並行執行！")
print("但注意：multiprocessing 有進程間通訊的開銷。")

### 1.3 GIL 的例外情況

GIL 在以下情況會被釋放：

1. **I/O 操作**：讀寫檔案、網路請求時會釋放 GIL
2. **C 擴展**：numpy、scipy 等 C 擴展在計算時可以釋放 GIL
3. **長時間的 C 函數呼叫**：如 numpy 的大矩陣運算

這就是為什麼 numpy 可以利用多核心！

---

## 第二部分：NumPy 的內建多執行緒

### 2.1 NumPy 的後端庫

NumPy 的矩陣運算使用底層的 BLAS/LAPACK 庫：

- **OpenBLAS**: 開源，大多數 Linux 發行版預設
- **Intel MKL**: Intel 優化版，在 Intel CPU 上通常更快
- **Apple Accelerate**: macOS 預設

這些庫已經**內建多執行緒支援**！

In [ ]:
# 檢查 numpy 的配置

print("NumPy configuration:")
print("=" * 50)
np.show_config()

In [ ]:
# 檢查環境變數

print("Thread-related environment variables:")
print("=" * 50)
vars_to_check = [
    'OMP_NUM_THREADS',      # OpenMP threads
    'OPENBLAS_NUM_THREADS', # OpenBLAS threads
    'MKL_NUM_THREADS',      # Intel MKL threads
    'NUMEXPR_NUM_THREADS',  # NumExpr threads
]

for var in vars_to_check:
    value = os.environ.get(var, 'not set')
    print(f"{var}: {value}")

In [ ]:
# 實驗：NumPy 多執行緒的效果

def benchmark_matmul(size, n_trials=5):
    """測量矩陣乘法的時間"""
    A = np.random.randn(size, size).astype(np.float64)
    B = np.random.randn(size, size).astype(np.float64)
    
    times = []
    for _ in range(n_trials):
        start = time.time()
        C = A @ B
        times.append(time.time() - start)
    
    return np.mean(times), np.std(times)

# 測試不同大小
sizes = [500, 1000, 2000, 3000]

print("Matrix multiplication benchmark:")
print("=" * 50)
print(f"{'Size':>10} {'Time (s)':>15} {'GFLOPS':>15}")
print("-" * 50)

for size in sizes:
    mean_time, std_time = benchmark_matmul(size)
    # FLOPS for matrix multiply: 2 * N^3
    flops = 2 * size**3
    gflops = flops / mean_time / 1e9
    print(f"{size:>10} {mean_time:>12.4f} ± {std_time:.4f} {gflops:>12.2f}")

print("\n觀察：NumPy 的矩陣乘法使用 BLAS，會自動利用多核心。")
print("如果你打開系統監視器，會看到多個 CPU 核心都在工作。")

In [ ]:
# 實驗：設定執行緒數量的影響
# 注意：這個設定必須在 import numpy 之前！這裡只是示範。

def show_thread_effect():
    """這個函數展示如何設定執行緒數
    
    實際使用時，應該在 Python 程式開頭設定：
    
    import os
    os.environ['OMP_NUM_THREADS'] = '4'  # 限制為 4 執行緒
    os.environ['OPENBLAS_NUM_THREADS'] = '4'
    os.environ['MKL_NUM_THREADS'] = '4'
    import numpy as np  # 在設定之後 import
    """
    print("設定執行緒數的方法：")
    print()
    print("方法 1: 環境變數（在 Python 啟動前）")
    print("  $ export OMP_NUM_THREADS=4")
    print("  $ python my_script.py")
    print()
    print("方法 2: 在程式開頭（import numpy 之前）")
    print("  import os")
    print("  os.environ['OMP_NUM_THREADS'] = '4'")
    print("  import numpy as np")
    print()
    print("方法 3: 使用 threadpoolctl（可以動態調整）")
    print("  from threadpoolctl import threadpool_limits")
    print("  with threadpool_limits(limits=4):")
    print("      # 這裡的計算只用 4 個執行緒")

show_thread_effect()

### 2.2 什麼操作會用到多執行緒？

**會用多執行緒的操作**（呼叫 BLAS/LAPACK）：
- 矩陣乘法：`A @ B`, `np.dot`, `np.matmul`
- 線性代數：`np.linalg.inv`, `np.linalg.svd`, `np.linalg.eig`
- 大的向量內積

**不會用多執行緒的操作**（純 NumPy）：
- Element-wise 運算：`A + B`, `A * B`, `np.sin(A)`
- Reduction：`np.sum`, `np.mean`, `np.max`
- 索引、reshape、transpose

In [ ]:
# 比較：BLAS 操作 vs 純 NumPy 操作

size = 2000
A = np.random.randn(size, size)
B = np.random.randn(size, size)

# 矩陣乘法（用 BLAS，多執行緒）
start = time.time()
for _ in range(5):
    C = A @ B
time_matmul = (time.time() - start) / 5

# Element-wise 乘法（純 NumPy，單執行緒）
start = time.time()
for _ in range(5):
    D = A * B
time_elementwise = (time.time() - start) / 5

# Sum（純 NumPy，單執行緒）
start = time.time()
for _ in range(5):
    s = np.sum(A)
time_sum = (time.time() - start) / 5

print(f"Matrix multiplication (BLAS): {time_matmul:.4f}s")
print(f"Element-wise multiplication:  {time_elementwise:.4f}s")
print(f"Sum reduction:                {time_sum:.4f}s")
print("\n觀察：矩陣乘法雖然計算量最大，但因為用了 BLAS 多執行緒，")
print("相對而言並沒有慢很多。")

---

## 第三部分：Benchmarking 方法

### 3.1 正確的 Benchmarking 原則

1. **Warmup**：第一次執行可能較慢（JIT、cache）
2. **多次執行**：取平均值和標準差
3. **控制變因**：固定隨機種子、輸入大小
4. **使用正確的計時器**：`time.perf_counter()` 比 `time.time()` 精確
5. **避免干擾**：關閉其他程式、使用專用環境

In [ ]:
class Timer:
    """簡單的計時器 context manager"""
    
    def __init__(self, name=""):
        self.name = name
        self.elapsed = 0
    
    def __enter__(self):
        self.start = time.perf_counter()
        return self
    
    def __exit__(self, *args):
        self.elapsed = time.perf_counter() - self.start
        if self.name:
            print(f"{self.name}: {self.elapsed:.4f}s")


def benchmark(func, *args, n_warmup=2, n_trials=10, **kwargs):
    """
    標準化的 benchmark 函數
    
    Parameters
    ----------
    func : callable
        要測試的函數
    n_warmup : int
        預熱次數
    n_trials : int
        正式測試次數
    
    Returns
    -------
    mean : float
        平均時間
    std : float
        標準差
    """
    # Warmup
    for _ in range(n_warmup):
        func(*args, **kwargs)
    
    # Benchmark
    times = []
    for _ in range(n_trials):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        times.append(time.perf_counter() - start)
    
    return np.mean(times), np.std(times), result


# 示範使用
def example_func(size):
    A = np.random.randn(size, size)
    return np.linalg.svd(A, compute_uv=False)

mean, std, _ = benchmark(example_func, 500)
print(f"SVD of 500x500 matrix: {mean:.4f} ± {std:.4f} seconds")

In [ ]:
# 使用 %timeit 魔術命令（Jupyter 專用）

A = np.random.randn(500, 500)
B = np.random.randn(500, 500)

print("使用 %%timeit 魔術命令：")
%timeit A @ B

### 3.2 Profiling：找出瓶頸

In [ ]:
import cProfile
import pstats
import io

def profile_code(func, *args, **kwargs):
    """
    Profile 一段程式碼並印出結果
    """
    pr = cProfile.Profile()
    pr.enable()
    
    result = func(*args, **kwargs)
    
    pr.disable()
    
    # 格式化輸出
    s = io.StringIO()
    ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
    ps.print_stats(15)  # 只印前 15 個
    print(s.getvalue())
    
    return result


def example_workflow():
    """一個包含多種操作的範例工作流程"""
    # 生成數據
    X = np.random.randn(1000, 100)
    
    # 正規化
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    
    # 計算 covariance
    cov = X.T @ X / len(X)
    
    # SVD
    U, S, Vt = np.linalg.svd(cov)
    
    # PCA projection
    X_proj = X @ Vt[:10].T
    
    return X_proj


print("Profiling example workflow:")
print("=" * 70)
result = profile_code(example_workflow)

### 3.3 記憶體 Profiling

In [ ]:
def estimate_memory(arr):
    """估計 numpy array 的記憶體使用"""
    bytes_per_element = arr.itemsize
    total_bytes = arr.nbytes
    
    # 轉換為人類可讀格式
    if total_bytes < 1024:
        return f"{total_bytes} B"
    elif total_bytes < 1024**2:
        return f"{total_bytes/1024:.2f} KB"
    elif total_bytes < 1024**3:
        return f"{total_bytes/1024**2:.2f} MB"
    else:
        return f"{total_bytes/1024**3:.2f} GB"


# 不同資料類型的記憶體使用
print("Memory usage by data type (1000x1000 matrix):")
print("=" * 50)

dtypes = [np.float16, np.float32, np.float64, np.int32, np.int64]
for dtype in dtypes:
    arr = np.zeros((1000, 1000), dtype=dtype)
    print(f"{str(dtype):20s}: {estimate_memory(arr):>10s}")

In [ ]:
# 計算你的 CNN 所需記憶體

def estimate_cnn_memory(batch_size, in_channels, height, width, 
                        layer_configs, dtype=np.float32):
    """
    估計 CNN forward pass 的記憶體需求
    
    Parameters
    ----------
    layer_configs : list of dict
        每一層的配置，例如：
        {'type': 'conv', 'out_channels': 32, 'kernel_size': 3}
        {'type': 'pool', 'kernel_size': 2}
    """
    bytes_per_element = np.dtype(dtype).itemsize
    total_bytes = 0
    
    H, W = height, width
    C = in_channels
    
    print(f"{'Layer':20s} {'Output Shape':25s} {'Memory':>15s}")
    print("-" * 60)
    
    # Input
    input_bytes = batch_size * C * H * W * bytes_per_element
    total_bytes += input_bytes
    print(f"{'Input':20s} {str((batch_size, C, H, W)):25s} {estimate_memory(np.zeros(1, dtype=dtype)):>15s}")
    
    for i, config in enumerate(layer_configs):
        if config['type'] == 'conv':
            out_C = config['out_channels']
            k = config.get('kernel_size', 3)
            p = config.get('padding', 0)
            s = config.get('stride', 1)
            
            H = (H + 2*p - k) // s + 1
            W = (W + 2*p - k) // s + 1
            
            # 參數記憶體
            param_bytes = (out_C * C * k * k + out_C) * bytes_per_element
            # 輸出記憶體
            output_bytes = batch_size * out_C * H * W * bytes_per_element
            # im2col 的記憶體（如果使用）
            im2col_bytes = batch_size * H * W * C * k * k * bytes_per_element
            
            layer_bytes = param_bytes + output_bytes + im2col_bytes
            total_bytes += layer_bytes
            C = out_C
            
            name = f"Conv{i+1}({out_C}ch, {k}x{k})"
            shape = (batch_size, out_C, H, W)
            
        elif config['type'] == 'pool':
            k = config.get('kernel_size', 2)
            H = H // k
            W = W // k
            
            output_bytes = batch_size * C * H * W * bytes_per_element
            layer_bytes = output_bytes
            total_bytes += layer_bytes
            
            name = f"Pool{i+1}({k}x{k})"
            shape = (batch_size, C, H, W)
        
        arr = np.zeros(shape, dtype=dtype)
        print(f"{name:20s} {str(shape):25s} {estimate_memory(arr):>15s}")
    
    print("-" * 60)
    total_arr = np.zeros(int(total_bytes / bytes_per_element), dtype=dtype)
    print(f"{'Total (estimated)':20s} {'-':25s} {estimate_memory(total_arr):>15s}")


# 範例：LeNet-style 網路
layers = [
    {'type': 'conv', 'out_channels': 32, 'kernel_size': 3, 'padding': 1},
    {'type': 'pool', 'kernel_size': 2},
    {'type': 'conv', 'out_channels': 64, 'kernel_size': 3, 'padding': 1},
    {'type': 'pool', 'kernel_size': 2},
    {'type': 'conv', 'out_channels': 128, 'kernel_size': 3, 'padding': 1},
]

print("\nMemory estimation for a small CNN:")
print("=" * 60)
estimate_cnn_memory(batch_size=32, in_channels=3, height=64, width=64,
                    layer_configs=layers)

---

## 第四部分：Threading vs Multiprocessing 總結

### 選擇指南

```
┌─────────────────────────────────────────────────────────────────┐
│                     Python 並行策略選擇                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌──────────────────┐     ┌──────────────────┐                 │
│  │   CPU-bound ?    │ Yes │ 用 numpy/scipy   │                 │
│  │  (計算密集型)     ├────►│ (自動多執行緒)    │                 │
│  └────────┬─────────┘     └──────────────────┘                 │
│           │ No                                                  │
│           ▼                                                     │
│  ┌──────────────────┐     ┌──────────────────┐                 │
│  │    I/O-bound ?   │ Yes │   threading      │                 │
│  │   (I/O 密集型)    ├────►│  (網路/檔案)      │                 │
│  └────────┬─────────┘     └──────────────────┘                 │
│           │ No                                                  │
│           ▼                                                     │
│  ┌──────────────────┐     ┌──────────────────┐                 │
│  │  純 Python 迴圈  │ Yes │  multiprocessing │                 │
│  │  無法向量化？    ├────►│   (用多進程)      │                 │
│  └────────┬─────────┘     └──────────────────┘                 │
│           │ No                                                  │
│           ▼                                                     │
│  ┌──────────────────┐                                          │
│  │   考慮向量化      │                                          │
│  │  或 numba/cython │                                          │
│  └──────────────────┘                                          │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# 總結實驗：不同策略的效能

def task_cpu_bound_python(n):
    """純 Python CPU 密集任務"""
    total = 0
    for i in range(n):
        total += i * i % 1000
    return total

def task_cpu_bound_numpy(size):
    """NumPy CPU 密集任務"""
    A = np.random.randn(size, size)
    B = np.random.randn(size, size)
    return A @ B

def task_io_bound():
    """I/O 密集任務（模擬）"""
    time.sleep(0.1)  # 模擬 I/O 等待
    return True

print("Performance Summary:")
print("=" * 70)

# 1. 純 Python CPU-bound
print("\n1. Pure Python CPU-bound (n=1,000,000):")
with Timer("   Sequential (2 tasks)"):
    task_cpu_bound_python(1_000_000)
    task_cpu_bound_python(1_000_000)

# 2. NumPy CPU-bound
print("\n2. NumPy CPU-bound (500x500 matmul, 10 times):")
with Timer("   Sequential"):
    for _ in range(10):
        task_cpu_bound_numpy(500)

# 3. I/O bound with threading
print("\n3. I/O bound (10 x 0.1s sleep):")
with Timer("   Sequential"):
    for _ in range(10):
        task_io_bound()

def threaded_io():
    threads = [threading.Thread(target=task_io_bound) for _ in range(10)]
    for t in threads:
        t.start()
    for t in threads:
        t.join()

with Timer("   Threaded (10 threads)"):
    threaded_io()

print("\n結論：")
print("- 純 Python 迴圈：用 multiprocessing 或改用 numpy")
print("- NumPy 矩陣運算：自動多執行緒，不需要額外處理")
print("- I/O 操作：threading 可以有效並行")

---

## 總結

### Python 效能要點

1. **GIL** 限制了純 Python 多執行緒的效能
   - 用 `multiprocessing` 繞過 GIL
   - 用 numpy/scipy 的 C 擴展自動釋放 GIL

2. **NumPy 多執行緒**
   - BLAS 操作（矩陣乘法、SVD 等）自動多執行緒
   - 可用環境變數控制執行緒數
   - Element-wise 操作不會多執行緒

3. **Benchmarking**
   - 使用 `time.perf_counter()` 而非 `time.time()`
   - 包含 warmup 階段
   - 多次測量取平均
   - 用 profiling 找出真正的瓶頸

### 下一步

- 學習向量化技巧（減少 Python 迴圈）
- 學習 im2col 把卷積轉成矩陣乘法（利用 BLAS 多執行緒）